# 5. Acoustic measurements

-----

In this final notebook we'll specify some of the **parameters that are widely used in acoustical measurements, more specifically in urban noise measurements, which is the main application of the developed device**.

We will call some of the functions introduced in the previous notebooks (which are included in `utils.py`) to obtain the different parameters, and **we'll develop new functionality to be able to easily calculate sound pressure levels with the device**.

## Sound pressure level

In acoustic measurements the variable that we're interested in is the instantaneous pressure, the dynamic variations in pressure around the atmospheric pressure. This value is **defined as the logarithmic relationship between this instantaneous pressure and the hearing threshold, which is about 20 μP**.

$$ SPL = 10 \log \left( \dfrac{P_{ef}^2}{\rho_0^2} \right) \ [dB_{SPL}] $$

$$ P_{ef} = \sqrt{ \dfrac{1}{T} \int_{o}^{T} p^2 (t) \ dt } $$

As we can see, $P_{ef}$ represents the RMS value of the instantaneous pressure.

## Integrating and averaging sound level meters

Since the instantaneous SPL will vary at every instant, we normally prefer to obtain a **measurement over a certain time interval, avearing the levels**. This can be obtained by either **exponential or linear averaging of the levels**. 

As we mentioned in the previous notebook the **sound level meters traditionally provided a way to ease the reading of levels by applying an RC integrating low-pass filter to the signal**.  The application of this procedure is known as **exponential averaging** of the sound pressure levels and the type of sound level meters that perform this operation are known as **averaging meters**. The exponential component is introduced to the SPL calculation as seen below.

$$ P_\tau = \sqrt{ \dfrac{1}{\tau} \int_{0}^{t} p^2 ( \theta ) \ e ^{ - \dfrac{t-\theta}{\tau}  } \  d\theta } $$

**Three time constants are used: Slow ($\tau$ = 1 s), Fast ($\tau$ = 125 ms) and Impulse ($\tau$ = 35 ms)**.

In the thesis we developed an **integrating-averaging meter**, which means that it can also linearly integrate the levels. 

In [ ]:
In the previous notebook we designed and apply these time-weighting filters, (excluding the Impulse filter, as it is not a requirement for sound level meters in the IEC 61672 standard)